In [2]:
import nltk
from nltk import word_tokenize, pos_tag
import os, os.path, sys
import glob
import numpy as np
import re
import pandas as pd
from nltk.stem import WordNetLemmatizer

def fetch(subTrees,mapper):               
    for eachTree in subTrees:
        if (type(eachTree)==nltk.tree.Tree):
            if eachTree.label() == "CompanyAndType":
                getCompanyAndType(eachTree,mapper,"Company")
            elif  eachTree.label() == "FirmAndType":
                getCompanyAndType(eachTree,mapper,"Firm")
            elif eachTree.label() == "RuleN": 
                
                ruleN(eachTree,mapper)
            elif eachTree.label() == "FirmAndCompany":
                firmAndCompany(eachTree,mapper)
    for eachFirm in mapper["firmCompany"]:
        tempParty = set()
        for eachCompany in mapper["firmCompany"][eachFirm]:
            if eachCompany in mapper["companyTypeMapper"]:
                tempParty = tempParty.union(mapper["companyTypeMapper"][eachCompany])
        for eachCompany in mapper["firmCompany"][eachFirm]:
            mapper["companyTypeMapper"][eachCompany] = tempParty
    
                
def getCompanyAndType(tree,TypeMapper,tag):
    
    treeLeaves = tree.leaves()

    companies = set()
    partyType = None
    for leave in treeLeaves:
        #print("leave : ",leave)
        if leave[1] == "PartyType":
            partyType = leave[0]
        elif leave[1] == tag:
            companies.add(leave[0])
    if partyType != None:
        if tag == "Company":
            
            for each in companies:
                if each not in TypeMapper["companyTypeMapper"]:
                    TypeMapper["companyTypeMapper"][each] = set()
                TypeMapper["companyTypeMapper"][each].add(partyType.lower())
        if tag == "Firm":
            partyType = partyType.lower()
            if partyType not in TypeMapper["firmTypeMapper"]:
                TypeMapper["firmTypeMapper"][partyType] = set()
            for each in companies:
                TypeMapper["firmTypeMapper"][partyType].add(each.replace("_firm__",""))

    #print("$$$$$$",TypeMapper,tree.leaves())
    return companies,partyType
      
    
def ruleN(tree,mapper):
    firms = list()
    for index in range(len(tree)):
        if(tree[index].label() == "CompanyAndType"):
            companies, partyType = getCompanyAndType(tree,mapper,"Company")
        else:
            firms = set([each[0] for each in tree.leaves() if each[1] == "Firm"])
    if len(companies) != 0 and len(firms) != 0:
        for eachCompany in companies:
            mapper["companyFirmMapper"][eachCompany] = mapper["companyFirmMapper"].setdefault(eachCompany,set()).union(firms)
        
        #mapper["firmTypeMapper"][partyType] = mapper["firmTypeMapper"].setdefault(partyType,set()).union(set(firms))
            
def firmAndCompany(tree,mapper):
    
    companies = set()
    leaves = tree.leaves()
    firms = set()
    for leave in leaves:
        if leave[1] == "Company":
            companies.add(leave[0])
        elif leave[1] == "Firm":
            firms.add(leave[0])
    if len(companies) != 0 and len(firms) != 0:
        for eachCompany in companies:
            mapper["companyFirmMapper"][eachCompany] = mapper["companyFirmMapper"].setdefault(eachCompany,set()).union(firms)
        for firm in firms:
            mapper["firmCompany"][firm] = mapper["firmCompany"].setdefault(firm,set()).union(companies)

def my_replace(regexObject):
    text = regexObject.group(0)
    if("{party" in text):
        index1 = 6
        index2 = text.find("_")    
        t = "_company__"+text[index1:index2]
    else:
        index1 = 5
        index2 = text.find("_")
        if index2 == -1 :
            index2 = text.find("}")
        t = "_firm__"+text[index1:index2]
    return t

def preProcess(data):
    text = data.strip().replace("v.","versus").replace(",","")
    regex = r"{party[A-Z0-9]{1,1}_}[^{]*{\/party[A-Z0-9]{1,1}_}|{firm[A-Z0-9]{1,1}_{0,1}}[^{]*{\/{0,1}firm[A-Z0-9]{1,1}_{0,1}[^}]*}"
    t = re.sub(regex, my_replace , text) #groups
    sentences = [each for each in t.split("\n") if each.strip() != ""]

    temp = [nltk.word_tokenize(sent) for sent in sentences]
    temp = [nltk.pos_tag(sent) for sent in temp]
    for eachSentence in temp:
        for index in range(len(eachSentence)):
            if (eachSentence[index][0].lower()) in partyTypesAnnotated:
                eachSentence[index] = (wnl.lemmatize(eachSentence[index][0].lower()),"PartyType")
            elif "_company__" in  eachSentence[index][0]:
                eachSentence[index] = (eachSentence[index][0],"Company")
            elif "_firm__" in  eachSentence[index][0]:
                eachSentence[index] = (eachSentence[index][0],"Firm")
            elif eachSentence[index][0] == "versus" or eachSentence[index][0].lower() == "against":
                eachSentence[index] = (eachSentence[index][0],"versus")
    grammar = r"""
                    C1:{(<FW>+)}
                    C1:{(<VBD>?<NN>)}
                    C1:   {(<DT>?<N.*>*)}
                    C1: {(<IN><:>?)}
                    C1 : {(<CC><C1>)}
                    R1:{(<C1>+<.>?)+}
                    CompanyN:{<IN>?<Company><R1>?}
                    CompanyN:{(<CompanyN><CC>?)+}
                    Againist:{<CompanyN><versus><CompanyN>}
                    PartyType:{<PartyType><:>?}
                    PartyType:{<IN>+<PartyType>}
                    PartyType:{<JJ>+<PartyType>}
                    CompanyAndType:{(<CompanyN><PartyType>)|(<PartyType><CompanyN>)}
                    firmN:{<R1>*<Firm><R1>*}
                    firmN:{(<firmN><CC>?)+}
                    FirmAndType:{<firmN><.>?<PartyType>}
                    FirmAndCompany:{<firmN><CompanyN>}
                    RuleN:{<CompanyAndType><firmN>}
                """
    cp = nltk.RegexpParser(grammar)
    result = []
    for each in temp:
        result.append(cp.parse(each))
        
        
    subTrees = []
    reqSet = {"RuleN","FirmAndType","FirmAndCompany","CompanyAndType"}#,"FirmN","CompanyN","versus","PartyType"}
    for each in (result):
        #for each node in tree
        for index in range(len(each)):
            #if tree then access its label
            if (type(each[index])==nltk.tree.Tree) and each[index].label() in reqSet:
                subTrees.append(each[index])

    return result,subTrees,temp,sentences


df = pd.read_csv("data.csv",dtype=str)
wnl = WordNetLemmatizer()
temp = list(df["Party type(s) - Annotated"])
partyTypesAnnotatedO = set()
for each in temp:
    t = each.split(",")
    for eachT in t:
        partyTypesAnnotatedO.add(eachT.strip())
        
partyTypesAnnotated = set() 
for each in partyTypesAnnotatedO:
    t = each+"s"
    partyTypesAnnotated.add(t)
    partyTypesAnnotated.add(each)
    
partyTypesAnnotated.add("appellant")
partyTypesAnnotated.add("appellants")


filePath  = "Opinions/Opinion{}.txt"

for i in range(1,11):
    fileName = filePath.format(i)
    data = open(fileName, 'r').read()
    result,subTrees,temp,sentences  = preProcess(data)
    mapper = {"companyTypeMapper":{},"firmTypeMapper":{},"companyFirmMapper":{},"firmCompany":{}}
    fetch(subTrees,mapper)
    #print("Opinion"+str(i)+") Processed")

    for index,row in (df[df.Opinion == str(i)]).iterrows():
        company = "_company__"+row["Party Letter"]
        if company in mapper["companyTypeMapper"]: 
            for party in mapper["companyTypeMapper"][company]:                
                if isinstance(df.at[index,"Party type(s) - Modeled"],float):
                    df.at[index,"Party type(s) - Modeled"] = party
                else:
                    df.at[index,"Party type(s) - Modeled"] +=  "," + party
                
                 
                if party in mapper["firmTypeMapper"]:
                    """
                    print("#########")
                    print(mapper["firmTypeMapper"][party], company,party)
                    if company in mapper["companyFirmMapper"]:
                        print(mapper["companyFirmMapper"][company])
                    else:
                        print("no companies mapped")
                    print("########")
                    """
                    temp = ",".join(mapper["firmTypeMapper"][party])
                    if isinstance(df.at[index,"Law firm(s) - Model"],float):
                        df.at[index,"Law firm(s) - Model"] = temp
                    else:
                        df.at[index,"Law firm(s) - Model"] += ", "+temp

        #print("companyFirmMapper",mapper["companyFirmMapper"])
        if company in mapper["companyFirmMapper"]:
            for f in mapper["companyFirmMapper"][company]:
                firmNo = f.replace("_firm__","")
                if isinstance(df.at[index,"Law firm(s) - Model"],float):
                    df.at[index,"Law firm(s) - Model"] = firmNo
                else:
                    df.at[index,"Law firm(s) - Model"] += ", "+firmNo

df

,Opinion,Party Letter,Party name,Party type(s) - Annotated,Law firm(s) - Annotated,Party type(s) - Modeled,Law firm(s) - Model
0,1,B,National Railroad Passenger Corp.,defendant,2,"defendant,plaintiff",2
1,1,G,Consolidated Rail Corp.,defendant,2,"defendant,plaintiff",2
2,2,A,"625 3RD ST. ASSOCS., L.P.",plaintiff,"1,2",plaintiff,"1,2"
3,2,B,ALLIANT CREDIT UNION,defendant,"3,4,5","defendant,plaintiff","5,4,3, 1,2"
4,3,A,ZENITH ELECTRONICS CORPORATION,"plaintiff, counter-defendant","1,2","counter-defendant,plaintiff","2, 1"
5,3,B,EXZEC INC.,"defendant, counter-claimant, counter-plaintiff","3,4","defendant,counter-claimant,counter-plaintiffs","4, 3"
6,3,C,CARROLL TOUCH INC,"defendant, counter-claimant, counter-plaintiff",5,"defendant,counter-claimant",5
7,4,D,Adamowicz Estates,plaintiff,1,plaintiff,1
8,5,A,ALLSTATE INSURANCE COMPANY,plaintiff,1,plaintiff,1
9,6,A,"SAMSUNG ELECTRONICS CO., LTD.",appellee,"1,2",appellees,2


In [9]:
df.to_csv("/Users/smanda1/Desktop/FinalResult.csv",index=False)

/Users/smanda1/Downloads/RavelDataChallenge


In [102]:
print(sentences[2])

For _company__A a Delaware corporation plaintiff: John M. Calimafde _firm__1 New York NY. Joel S. Feldman _firm__2 Chicago IL.


[('Adam', 'NNP'), ('Keats', 'NNP'), ('_firm__2', 'Firm'), ('San', 'NNP'), ('Francisco', 'NNP'), ('CA', 'NNP'), ('for', 'IN'), ('defendant', 'PartyType'), ('.', '.')]


In [ ]:
df.to_csv("modeledData.csv",index=False)

In [ ]:
"R1:   {<DT>?<N.*>*}
firmN:{<R1>*<Firm><R1>*}
PartyType:{<PartyType><:>?}
PartyType:{<IN>+<PartyType><.>?<,>?}
FirmAndType:{<firmN><,>?<.>?<PartyType><,>?}

In [3]:
result[2].draw()

[('November', 'NNP'), ('7', 'CD'), ('2007', 'CD'), ('.', '.')]

In [105]:
data = "For {partyA_}ZENITH ELECT CORPORATION{/partyA_}, a Delaware corporation, plaintiff: John M. Calimafde, {firm1_}Hopgood, Calinafde, Kalil, Blaustein & Judlowe{/firm1_}, New York, NY. Joel S. Feldman, {firm2_}Sachnoff & Weaver, Ltd.{/firm2_}, Chicago, IL."

result,subTrees,tokens,sentences = preProcess(data)



For
_company__A
a
Delaware
corporation
plaintiff
:
John
M.
Calimafde
_firm__1
New
York
NY
.
Joel
S.
Feldman
_firm__2
Chicago
IL
.


In [87]:
sentences
#PartyType:{<IN>+<PartyType><.>?<,>?}
#                    R1:   {<DT>?<N.*>*}
#                    FirmAndType:{<firmN><,>?<.>?<PartyType><,>?}
#                    firmN:{<R1>*<Firm><R1>*}


['For _company__A a Delaware corporation plaintiff: John M. Calimafde _firm__1 New York NY. Joel S. Feldman _firm__2 Chicago IL.']

In [88]:
tokens

[[('For', 'IN'),
  ('_company__A', 'Company'),
  ('a', 'DT'),
  ('Delaware', 'NNP'),
  ('corporation', 'NN'),
  ('plaintiff', 'PartyType'),
  (':', ':'),
  ('John', 'NNP'),
  ('M.', 'NNP'),
  ('Calimafde', 'NNP'),
  ('_firm__1', 'Firm'),
  ('New', 'NNP'),
  ('York', 'NNP'),
  ('NY', 'NNP'),
  ('.', '.'),
  ('Joel', 'NNP'),
  ('S.', 'NNP'),
  ('Feldman', 'NNP'),
  ('_firm__2', 'Firm'),
  ('Chicago', 'NNP'),
  ('IL', 'NNP'),
  ('.', '.')]]

In [89]:
result[0].draw()

In [28]:
#lambda , regexObjects

In [95]:
from nltk import Tree
from nltk.draw.util import CanvasFrame
from nltk.draw import TreeWidget

cf = CanvasFrame()
tc = TreeWidget(cf.canvas(),result[0])
tc['node_font'] = 'arial 14 bold'
tc['leaf_font'] = 'arial 14'
tc['node_color'] = '#005990'
tc['leaf_color'] = '#3F8F57'
tc['line_color'] = '#175252'
cf.add_widget(tc,10,10) # (10,10) offsets
cf.print_to_file('tree.ps')
cf.destroy()

In [ ]:
                    C1:{<FW>+}
                    C1:{<VBD>?<NN>}
                    C1:   {<DT>?<N.*>*}
                    IN: {<IN><:>}
                    C1 : {<CC><C1>}
                    R1:{<C1>+}
                    R1 : {<IN>?<R1><.>?}
                    PartyType:{<PartyType><:>?}
                    CompanyN:{<IN>?<Company><,>?<R1>?}
                    CompanyN:{<CompanyN><CC>?<CompanyN><CC>?<CompanyN>?}
                    Againist:{<CompanyN><versus><CompanyN>}
                    PartyType:{<IN>+<PartyType><.>?<,>?}
                    PartyType:{<JJ>+<PartyType>}
                    CompanyAndType:{<PartyType><CompanyN>}
                    CompanyAndType:{<CompanyN><,>?<PartyType><,>?<.>?}
                    firmN:{<R1>*<Firm><R1>*}
                    firmN:{<firmN><CC>?<firmN>?<CC>?<firmN>?}
                    FirmAndType:{<firmN><,>?<.>?<PartyType><,>?}
                    FirmAndCompany:{<firmN><CompanyN>}
                    RuleN:{<CompanyAndType><firmN>}      